# Import packages

In [4]:
import math, os, collections
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from evaluation import Evaluation
from db_connection.utils import get_conn, get_data_start_dt
from utils import load_w103, load_w106, load_cust, cust_process
from mlaas_tools.config_build import config_set

In [6]:
## Configure env
if not os.path.isfile('config.ini'):
    config_set()

In [7]:
## Load db connection
rawdata_conn = get_conn('edu')

read key file
login as edu-cytsao


In [9]:
rawdata_conn = get_conn('edu')
# interaction train w103
w103_df = load_w103('2019-01-31', rawdata_conn, span=18)
w103_df['wm_prod_code'].value_counts()

read key file
login as edu-cytsao


AG25    83106
KK32    75595
CC03    54323
AA39    48277
CC88    40423
        ...  
2905        1
5009        1
2207        1
PP52        1
8928        1
Name: wm_prod_code, Length: 2263, dtype: int64

In [20]:
w103_df.groupby(["cust_no","wm_prod_code"])['wm_prod_code'].value_counts()[:5]#.index.to_list()

cust_no                   wm_prod_code  wm_prod_code
++++2rDzc1I9amUGBhmxLA==  5821          5821             1
++1QLn8bCFzxWN8vQYBuRg==  NN18          NN18            32
++2WLG2lyrgp9jhLkCFEKg==  2509          2509            18
                          AG74          AG74            18
                          WW06          WW06            18
Name: wm_prod_code, dtype: int64

In [17]:
w103_df.groupby(["cust_no","wm_prod_code"]).head()

,cust_no,wm_prod_code,txn_dt,txn_amt,dta_src,deduct_cnt,etl_dt
0,0016boFyZaRofUFAYfXYTA==,2535,2018-04-16,5800000.0,,0.0,2018-11-12
1,008d2RbDuEfP8n5Dc/t20Q==,5906,2017-12-11,525944.0,,0.0,2018-09-25
2,00b4qBwr1FZJW6zTYLYyQA==,KK41,2017-08-07,3993648.0,,0.0,2018-09-25
3,00CC0bunwJX3jgJwycvHvQ==,2446,2017-09-08,11600000.0,,0.0,2018-09-25
4,00EEcEgHkpb6NGcolnAEog==,UF56,2017-09-28,1392000.0,,0.0,2018-09-25
...,...,...,...,...,...,...,...
1950002,ZzzqJZTdev3aK41clf5i1w==,FCA3,2018-06-21,876670.0,,0.0,2018-11-12
1950003,ZzzqJZTdev3aK41clf5i1w==,FE41,2018-04-12,846510.0,,0.0,2018-09-25
1950004,ZzzqJZTdev3aK41clf5i1w==,NN83,2018-03-28,725290.0,,0.0,2018-09-25
1950005,ZzzqJZTdev3aK41clf5i1w==,NN83,2017-09-12,402636.0,,0.0,2018-09-25


In [18]:
print(len(w103_df), len(w103_df.cust_no.unique()))

1950007 72389


In [15]:
w103_df.groupby(["cust_no","wm_prod_code"], group_keys=False).apply(lambda x: x.sort_values(["deduct_cnt"],ascending=True).head(1))

,cust_no,wm_prod_code,txn_dt,txn_amt,dta_src,deduct_cnt,etl_dt
90556,++++2rDzc1I9amUGBhmxLA==,5821,2017-11-27,1113229.0,,0.0,2018-09-25
54929,++1QLn8bCFzxWN8vQYBuRg==,NN18,2017-08-08,29000.0,,3.0,2018-09-25
95384,++2WLG2lyrgp9jhLkCFEKg==,2509,2017-08-14,29000.0,,122.0,2018-09-25
95394,++2WLG2lyrgp9jhLkCFEKg==,AG74,2017-08-17,29000.0,,124.0,2018-09-25
95380,++2WLG2lyrgp9jhLkCFEKg==,WW06,2017-08-17,29000.0,,116.0,2018-09-25
...,...,...,...,...,...,...,...
1949729,zzvYlT/G3nOSg2SdheESWg==,AT07,2018-02-07,1021728.0,A0,0.0,2018-09-25
1949723,zzvYlT/G3nOSg2SdheESWg==,AV03,2018-02-12,17011.0,,1.0,2018-09-25
1949731,zzvYlT/G3nOSg2SdheESWg==,BB39,2017-11-02,870000.0,A0,0.0,2018-09-25
1949739,zzvYlT/G3nOSg2SdheESWg==,FE41,2017-11-02,870000.0,,0.0,2018-09-25
